# Predict the stock market with news

In [8]:
# Set up the environment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import pandas as pd
import numpy as np
import gensim.models
from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.preprocessing import Tokenizer, text_to_word_sequence                                                                                                      from sklearn.preprocessing import FunctionTransformer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re

# NLP libraries
from textblob import TextBlob

In [42]:
import sys
sys.path.append('../')
from STOCK_PREDICT.df import get_data
df_djia, df_news = get_data(1989)

FileNotFoundError: [Errno 2] File b'/Users/Jean-Phi-ben/code/Jean-Phi-ben/STOCK_PREDICT/data/datasets-129-792900-upload_DJIA_table.csv' does not exist: b'/Users/Jean-Phi-ben/code/Jean-Phi-ben/STOCK_PREDICT/data/datasets-129-792900-upload_DJIA_table.csv'

In [15]:
news.iloc[100,26]

'b\'The Arab Conscience: A 40 minute song performed by many major Arab pop singers. "The Woodstock of this generation in the Arab world", according to Juan Cole.\''

## Data cleaning

## Feature creation with Textblob

References : 
https://textblob.readthedocs.io/en/dev/quickstart.html#create-a-textblob


#### Textblobbing

In [9]:
#Textblob
wiki = TextBlob(df['combined'][0])

# Subjectivity and polarity
polarity = wiki.sentiment[0]
subjectivity = wiki.sentiment[1]
polarity, subjectivity

NameError: name 'df' is not defined

#### Textblob function

In [ ]:
def polarity(text):
    blob = TextBlob(text)
    polarity = blob.sentiment[0]
    return polarity

def subjectivity(text):
    blob = TextBlob(text)
    subjectivity = blob.sentiment[1]
    return subjectivity

In [ ]:
# Feature creation
df['polarity'] = df['combined'].apply(polarity)
df['subjectivity'] = df['combined'].apply(subjectivity)

# Merge data

In [16]:
trade.tail()

Date          Open          High           Low         Close  \
1984  2008-08-14  11532.070312  11718.280273  11450.889648  11615.929688   
1985  2008-08-13  11632.809570  11633.780273  11453.339844  11532.959961   
1986  2008-08-12  11781.700195  11782.349609  11601.519531  11642.469727   
1987  2008-08-11  11729.669922  11867.110352  11675.530273  11782.349609   
1988  2008-08-08  11432.089844  11759.959961  11388.040039  11734.320312   

         Volume     Adj Close  
1984  159790000  11615.929688  
1985  182550000  11532.959961  
1986  173590000  11642.469727  
1987  183190000  11782.349609  
1988  212830000  11734.320312

In [17]:
news.head()

Date  Label                                               Top1  \
0  2008-08-08      0  b"Georgia 'downs two Russian warplanes' as cou...   
1  2008-08-11      1  b'Why wont America and Nato help us? If they w...   
2  2008-08-12      0  b'Remember that adorable 9-year-old who sang a...   
3  2008-08-13      0  b' U.S. refuses Israel weapons to attack Iran:...   
4  2008-08-14      1  b'All the experts admit that we should legalis...   

                                                Top2  \
0            b'BREAKING: Musharraf to be impeached.'   
1        b'Bush puts foot down on Georgian conflict'   
2                 b"Russia 'ends Georgia operation'"   
3  b"When the president ordered to attack Tskhinv...   
4  b'War in South Osetia - 89 pictures made by a ...   

                                                Top3  \
0  b'Russia Today: Columns of troops roll into So...   
1  b"Jewish Georgian minister: Thanks to Israeli ...   
2  b'"If we had no sexual harassment we would hav...   
3  b' Israel clears troops who killed Reuters cam...   
4  b'Swedish wrestler Ara Abrahamian throws away ...   

                                                Top4  \
0  b'Russian tanks are moving towards the capital...   
1  b'Georgian army flees in disarray as Russians ...   
2  b"Al-Qa'eda is losing support in Iraq because ...   
3  b'Britain\'s policy of being tough on drugs is...   
4  b'Russia exaggerated the death toll in South O...   

                                                Top5  \
0  b"Afghan children raped with 'impunity,' U.N. ...   
1      b"Olympic opening ceremony fireworks 'faked'"   
2  b'Ceasefire in Georgia: Putin Outmaneuvers the...   
3  b'Body of 14 year old found in trunk; Latest (...   
4  b'Missile That Killed 9 Inside Pakistan May Ha...   

                                                Top6  \
0  b'150 Russian tanks have entered South Ossetia...   
1  b'What were the Mossad with fraudulent New Zea...   
2  b'Why Microsoft and Intel tried to kill the XO...   
3  b'China has moved 10 *million* quake survivors...   
4  b"Rushdie Condemns Random House's Refusal to P...   

                                                Top7  \
0  b"Breaking: Georgia invades South Ossetia, Rus...   
1  b'Russia angered by Israeli military sale to G...   
2  b'Stratfor: The Russo-Georgian War and the Bal...   
3  b"Bush announces Operation Get All Up In Russi...   
4  b'Poland and US agree to missle defense deal. ...   

                                                Top8  ...  \
0  b"The 'enemy combatent' trials are nothing but...  ...   
1  b'An American citizen living in S.Ossetia blam...  ...   
2  b"I'm Trying to Get a Sense of This Whole Geor...  ...   
3             b'Russian forces sink Georgian ships '  ...   
4  b'Will the Russians conquer Tblisi? Bet on it,...  ...   

                                               Top16  \
0  b'Georgia Invades South Ossetia - if Russia ge...   
1  b'Israel and the US behind the Georgian aggres...   
2  b'U.S. troops still in Georgia (did you know t...   
3                      b'Elephants extinct by 2020?'   
4  b'Bank analyst forecast Georgian crisis 2 days...   

                                               Top17  \
0                b'Al-Qaeda Faces Islamist Backlash'   
1  b'"Do not believe TV, neither Russian nor Geor...   
2       b'Why Russias response to Georgia was right'   
3  b'US humanitarian missions soon in Georgia - i...   
4  b"Georgia confict could set back Russia's US r...   

                                               Top18  \
0  b'Condoleezza Rice: "The US would not act to p...   
1  b'Riots are still going on in Montreal (Canada...   
2  b'Gorbachev accuses U.S. of making a "serious ...   
3             b"Georgia's DDOS came from US sources"   
4  b'War in the Caucasus is as much the product o...   

                                               Top19  \
0  b'This is a busy day:  The European Union has ...   
1    b'China to overtake US as largest manufac

In [4]:
#merge dataset
news['Date'] = pd.to_datetime(news['Date'])
trade['Date'] = pd.to_datetime(trade['Date'])

In [5]:
data = news.merge(trade, on='Date')

In [20]:
data

Date  Label                                               Top1  \
0    2008-08-08      0  b"Georgia 'downs two Russian warplanes' as cou...   
1    2008-08-11      1  b'Why wont America and Nato help us? If they w...   
2    2008-08-12      0  b'Remember that adorable 9-year-old who sang a...   
3    2008-08-13      0  b' U.S. refuses Israel weapons to attack Iran:...   
4    2008-08-14      1  b'All the experts admit that we should legalis...   
5    2008-08-15      1  b"Mom of missing gay man: Too bad he's not a 2...   
6    2008-08-18      0  b'In an Afghan prison, the majority of female ...   
7    2008-08-19      0  b"Man arrested and locked up for five hours af...   
8    2008-08-20      1  b'Two elderly Chinese women have been sentence...   
9    2008-08-21      1  b"British resident held in Guantanamo Bay wins...   
10   2008-08-22      1  b'Syria says its ready to put a Russian missil...   
11   2008-08-25      0  b"N Korea's Kim died in 2003; replaced by look...   
12   2008-08-26      1  b'North Korea halts denuclearisation after US ...   
13   2008-08-27      1  b'Photos of a 15-year-old Iraqi suicide bomber...   
14   2008-08-28      1  b"Military help for Georgia is a 'declaration ...   
15   2008-08-29      0  b'Russian Prime Minister Vladimir Putin has ac...   
16   2008-09-02      0  b"A girl filmed Israeli soldiers' brutality. S...   
17   2008-09-03      1  b'Poland Legalization of Marijuana - two majou...   
18   2008-09-04      0  b'Security guards beat man at soccer (football...   
19   2008-09-05      1  b'In Jordan, the honor killings that are not: ...   
20   2008-09-08      1  b'A French judge has ordered two branches of S...   
21   2008-09-09      0  b'I love the world: Thai Prime Minister Samak ...   
22   2008-09-10      1  b'Dutch to ban burkas when picking children up...   
23   2008-09-11      1  b'American intelligence confirms that the late...   
24   2008-09-12      0     b"OPEC finished??  Saudi Arabia says 'LATER!'"   
25   2008-09-15      0  b'Did Saakashvili Lie?: The West Begins to Dou...   
26   2008-09-16      1  b"Pakistani Tribesmen Representing Half A Mill...   
27   2008-09-17      0    b'Attack on US embassy in Yemen leaves 16 dead'   
28   2008-09-18      1      b'6,200 babies sick: Milk scandal spreading '   
29   2008-09-19      1  b'Brits take a stand - ban short-selling of fi...   
...         ...    ...                                                ...   
1959 2016-05-20      1  LinkedIn just revealed that the 2012 breach th...   
1960 2016-05-23      0  'It keeps happening,' rape survivor says: Prot...   
1961 2016-05-24      1  Colombia has fired more than 1000 police offic...   
1962 2016-05-25      1  Adidas shoe manufacture returns to Germany, fu...   
1963 2016-05-26      0  Greek news sites are reporting that Aristotle'...   
1964 2016-05-27      1  Thailand has shut down 10 popular diving sites...   
1965 2016-05-31      0  Every reference to the Great Barrier Reef remo...   
1966 2016-06-01      1  Forty tiger cub bodies have been found in a fr...   
1967 2016-06-02      1           Germany recognizes the Armenian Genocide   
1968 2016-06-03      0  A seven-year-old boy who went missing in bear-...   
1969 2016-06-06      1  Nearly 60% of Qatar population live in 'labour...   
1970 2016-06-07      1  Five Men in India Have Been Convicted for the ...   
1971 2016-06-08      1  UN Removes Saudi Arabia From Human Rights Blac...   
1972 2016-06-09      0  Vandals Destroy 8,000-Yr-Old Aboriginal Artwor...   
1973 2016-06-10      0  U.N. Chief Admits He Removed Saudi Arabia From...   
1974 2016-06-13      0  A Dutch woman who alleged she was raped after ...   
1975 2016-06-14      0  Gay Chinese man sues mental hospital for tryin...   
1976 2016-06-15      0  ISIS Twitter accounts have been hijacked with ...   
1977 2016-06-16      1  Decriminalisation Call As 'War On Drugs' Fails...   
1978 2016-06-17      0               MP Jo Cox dead after shooting attack   
1979 20

In [21]:
data['Top1'][8]

"b'Two elderly Chinese women have been sentenced to a year of re-education through labor after they sought a permit to demonstrate in an official Olympic protest area.'"

# Add target column and % change column

In [25]:
# get percentage change
data['change'] = data['Open'].pct_change()
# remove first row


NameError: name 'data' is not defined

In [24]:
data= data[1:].reset_index(drop=True)

NameError: name 'data' is not defined

In [16]:
data.drop('index', axis=1, inplace=True)

In [17]:
data.head()

Date  Label                                               Top1  \
0 2008-08-13      0  b' U.S. refuses Israel weapons to attack Iran:...   
1 2008-08-14      1  b'All the experts admit that we should legalis...   
2 2008-08-15      1  b"Mom of missing gay man: Too bad he's not a 2...   
3 2008-08-18      0  b'In an Afghan prison, the majority of female ...   
4 2008-08-19      0  b"Man arrested and locked up for five hours af...   

                                                Top2  \
0  b"When the president ordered to attack Tskhinv...   
1  b'War in South Osetia - 89 pictures made by a ...   
2  b"Russia: U.S. Poland Missile Deal Won't Go 'U...   
3          b"Little girl, you're not ugly; they are"   
4  b'The US missile defence system is the magic p...   

                                                Top3  \
0  b' Israel clears troops who killed Reuters cam...   
1  b'Swedish wrestler Ara Abrahamian throws away ...   
2  b"The government has been accused of creating ...   
3  b"Pakistan's Musharraf to Resign, Leave the Co...   
4  b'Schrder lambasted for blaming Russian confli...   

                                                Top4  \
0  b'Britain\'s policy of being tough on drugs is...   
1  b'Russia exaggerated the death toll in South O...   
2  b'The Italian government has lashed out at an ...   
3  b'Tornado throws a bus in Poland, captured by ...   
4  b'Officials: 10 French soldiers killed near Ka...   

                                                Top5  \
0  b'Body of 14 year old found in trunk; Latest (...   
1  b'Missile That Killed 9 Inside Pakistan May Ha...   
2  b'Gorbachev: Georgia started conflict in S. Os...   
3  b"Britain's terror laws have left me and my fa...   
4  b'These ten laws make China a totalitarian was...   

                                                Top6  \
0  b'China has moved 10 *million* quake survivors...   
1  b"Rushdie Condemns Random House's Refusal to P...   
2  b"China fakes more than your girlfriend; 'Ethn...   
3               b"Iran 'fires satellite into space'"   
4                       b'Russia seizes US vehicles'   

                                                Top7  \
0  b"Bush announces Operation Get All Up In Russi...   
1  b'Poland and US agree to missle defense deal. ...   
2  b"The UN's criticism of freedom of expression ...   
3  b'Rights of Non-Muslims restricted by new Mald...   
4  b"Muslims are only 4% of Denmark's 5.4 million...   

                                                Top8  ...  \
0             b'Russian forces sink Georgian ships '  ...   
1  b'Will the Russians conquer Tblisi? Bet on it,...  ...   
2  b'Russian general threatens nuclear strike on ...  ...   
3  b'Tour of Tskhinvali undercuts Russian version...  ...   
4  b'Taliban Forces Kill 10 French Soldiers and R...  ...   

                                               Top23  \
0  b' Quarter of Russians blame U.S. for conflict...   
1  b'Russia: World  "can forget about" Georgia\'s...   
2  b'Bush Tells Putin: This Aggression Will Not S...   
3  b'Putin has given us an order that everyone mu...   
4  b'Taliban Forces Kill 10 French Soldiers and R...   

                                               Top24  \
0  b'Georgian president  says US military will ta...   
1  b'Darfur rebels accuse Sudan of mounting major...   
2          b'Georgia is all about the oil pipelines'   
3  b'National DNA database grows on the genes of ...   
4      b'Pakistan is more democratic than America. '   

                                               Top25          Open  \
0  b'2006: Nobel laureate Aleksander Solzhenitsyn...  11632.809570   
1  b'Philippines : Peace Advocate say Muslims nee...  11532.070312   
2  b'Rivals say they plan to remove Georgian pres...  11611.209961   
3         b'Mayor Asks Ugly Women To Visit His Town'  11659.650391   
4               b'Blaze engulfs Egyptian parliament'  11478.089844   

           High           Low         Close     Volume     Adj Close    change 

In [18]:
# Get categorical data
def categorical(x):
    if x > 0:
        x = 1
    else:
        x = 0
    return x

In [19]:
data['target'] = data['change'].apply(categorical)



In [20]:
# Combine the top 25 daily news into 1 column
cols = data.columns[2:27]
data['combined'] = data[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [27]:
data['combined'][10]

'b\'Syria says its ready to put a Russian missile system on its soil as a counterweight to U.S. plans to deploy a missile shield in Poland and the Czech Republic\'_b\'"Superclass"- new book - over the past several decades a "global elite" has emerged whose connections to each other have become more significant than their ties to their home nations and governments.\'_b\'Georgia did not believe Russia would respond to its offensive in South Ossetia and was completely unprepared for the counter-attack, the deputy defence minister has admitted.\'_b\'Ossetian leader admits conducting ethnic cleansing\'_b\'Report: Push for Iraq War Preceded Intelligence FIndings\'_b"Russia Capitalizes on Bush\'s New World Disorder"_b\'Americans including James Powerderly arrested at Olympics as China Clamp Downs on Pro-Tibetan Groups--\'_b\'Prohibited Places FOIA request refused by UK Ministry of Justice on the grounds of cost / time\'_b\'An acute water shortage in the West Bank has become so severe that mos

# Naive_base avec model avec column combined

In [27]:
df = data[['Date', 'combined']]

In [28]:
df.head()

Date                                           combined
0 2008-08-13  b' U.S. refuses Israel weapons to attack Iran:...
1 2008-08-14  b'All the experts admit that we should legalis...
2 2008-08-15  b"Mom of missing gay man: Too bad he's not a 2...
3 2008-08-18  b'In an Afghan prison, the majority of female ...
4 2008-08-19  b"Man arrested and locked up for five hours af...

In [23]:
def clean(text):
    for punctuation in string.punctuation:
        review1 = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
#     if re.search(r'^b\s', text):
#         text = text.replace("b ","")
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return " ".join(lemmatized)

df['cleaned'] = df['combined'].apply(clean)

NameError: name 'df' is not defined

In [28]:
import sys
sys.path.append('../')
from STOCK_PREDICT.df import get_data
df_djia, df_news = get_data()

ModuleNotFoundError: No module named 'df'

In [ ]:
USERNAME='Jean-Phi-ben'
def get_data(nrows):
    path1=f'/Users/{USERNAME}/code/{USERNAME}/STOCK_PREDICT/data/datasets-129-792900-upload_DJIA_table.csv'
    path2=f'/Users/{USERNAME}/code/{USERNAME}/STOCK_PREDICT/data/Combined_News_DJIA.csv'
    df_djia = pd.read_csv(path1, nrows=nrows)
    df_news = pd.read_csv(path2,nrows=nrows)
    return df_djia, df_news

In [13]:
pwd

'/home/jp/code/STOCK_PREDICT/Notebooks'

In [35]:
import sys
sys.path.append('../')
from STOCK_PREDICT.df import get_data
df_djia, df_news = get_data(1989)

FileNotFoundError: [Errno 2] File b'/Users/Jean-Phi-ben/code/Jean-Phi-ben/STOCK_PREDICT/data/datasets-129-792900-upload_DJIA_table.csv' does not exist: b'/Users/Jean-Phi-ben/code/Jean-Phi-ben/STOCK_PREDICT/data/datasets-129-792900-upload_DJIA_table.csv'

In [6]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jp/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True